In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
#Import other necessary model libraries, for this example, using Logistic Regression

In [2]:
train_data = pd.read_csv("data/train_dataset.csv")

In [3]:
train_data.tail()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,...,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
10495,1438655,4G,Smartphone,MALE,Kalutara,40-50,64,2088.13,479.0,0.346959,...,29.866667,0.0,7.0,14.571429,8.017054,0.0,0.0,NaN,NaN,PKG6
10496,7827264,4G,Smartphone,FEMALE,Colombo,<20,6,2053.50,23168.0,0.219985,...,25.366667,0.0,3.0,10.861111,5.151029,0.0,0.0,580.0,3.0,PKG1
10497,1433957,4G,Smartphone,FEMALE,Colombo,30-40,69,2827.33,4553.0,4.831711,...,0.000000,0.0,0.0,NaN,0.000000,0.0,1.0,NaN,NaN,PKG6
10498,8494507,4G,Smartphone,MALE,Kandy,60-70,233,645.43,2069.0,4.722031,...,18.650000,0.0,18.0,5.887963,4.956822,0.0,1.0,NaN,NaN,PKG1
10499,3569439,3G,Smartphone,MALE,Puttalam,40-50,33,498.20,536.0,0.003048,...,4.366667,0.0,7.0,2.228571,1.096541,0.0,0.0,NaN,NaN,PKG1


In [4]:
train_data['next_month_plan'] = train_data['next_month_plan'].astype('category')

In [5]:
train_data['next_month_plan'].unique()

['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']
Categories (8, object): ['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']

In [6]:
labels_x = ['PKG0','PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7', 'PKG8']

In [7]:
le = preprocessing.LabelEncoder()

In [8]:
le.fit(labels_x)

LabelEncoder()

In [9]:
le.classes_

array(['PKG0', 'PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7',
       'PKG8'], dtype='<U4')

In [10]:
train_data['encoded_class_labels'] = le.transform(train_data['next_month_plan'])

In [11]:
train_data.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,...,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan,encoded_class_labels
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317.0,2.773958,...,0.0,27.0,25.099383,34.759221,0.0,1.0,NaN,NaN,PKG2,2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365.0,3.079438,...,0.0,15.0,5.071111,6.779644,0.0,0.0,600.0,2.0,PKG1,1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581.0,48.193713,...,0.0,0.0,NaN,0.000000,0.0,0.0,0.0,1.0,PKG6,6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071.0,22.437842,...,0.0,0.0,NaN,0.000000,0.0,0.0,1110.0,2.0,PKG4,4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736.0,53.302092,...,0.0,23.0,11.256522,16.224071,0.0,1.0,1350.0,2.0,PKG4,4


In [12]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   primary_identifier           10500 non-null  int64   
 1   device_type                  10477 non-null  object  
 2   device_category              10470 non-null  object  
 3   gender                       10365 non-null  object  
 4   district_name                10500 non-null  object  
 5   age_group                    10365 non-null  object  
 6   network_stay                 10500 non-null  int64   
 7   average_monthly_bill_amount  10500 non-null  float64 
 8   dusage_sum                   10500 non-null  float64 
 9   dusage_min                   10500 non-null  float64 
 10  dusage_max                   10500 non-null  float64 
 11  dusage_avg                   9642 non-null   float64 
 12  dusage_days                  10500 non-null  float64 
 13  d

In [13]:
train_data = train_data.dropna(subset = ['device_type', 'device_category', 'gender', 'age_group'], axis = 0)

In [14]:
train_data = train_data.drop(columns = ['next_month_plan', 'vusage_onnet_avg'])

In [15]:
train_data['dusage_avg'] = train_data['dusage_avg'].fillna(value = 0)
train_data['vusage_offnet_avg'] = train_data['vusage_offnet_avg'].fillna(value = 0)
train_data['add_on_tot_rental'] = train_data['add_on_tot_rental'].fillna(value = 0)
train_data['add_on_count'] = train_data['add_on_count'].fillna(value = 0)

In [16]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10337 entries, 0 to 10499
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10337 non-null  int64  
 1   device_type                  10337 non-null  object 
 2   device_category              10337 non-null  object 
 3   gender                       10337 non-null  object 
 4   district_name                10337 non-null  object 
 5   age_group                    10337 non-null  object 
 6   network_stay                 10337 non-null  int64  
 7   average_monthly_bill_amount  10337 non-null  float64
 8   dusage_sum                   10337 non-null  float64
 9   dusage_min                   10337 non-null  float64
 10  dusage_max                   10337 non-null  float64
 11  dusage_avg                   10337 non-null  float64
 12  dusage_days                  10337 non-null  float64
 13  dusage_stddev   

In [17]:
train_data.columns

Index(['primary_identifier', 'device_type', 'device_category', 'gender',
       'district_name', 'age_group', 'network_stay',
       'average_monthly_bill_amount', 'dusage_sum', 'dusage_min', 'dusage_max',
       'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
       'vusage_onnet_max', 'vusage_onnet_min', 'vusage_onnet_days',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count',
       'encoded_class_labels'],
      dtype='object')

In [18]:
data_cat_cols = ['device_type', 'device_category', 'gender', 'district_name', 'age_group']

In [19]:
data_quan_cols = ['network_stay','average_monthly_bill_amount', 'dusage_sum', 
                 'dusage_min', 'dusage_max','dusage_avg', 'dusage_days', 
                 'dusage_stddev', 'vusage_onnet_sum','vusage_onnet_max', 
                 'vusage_onnet_min', 'vusage_onnet_days','vusage_onnet_stddev', 
                 'vusage_offnet_sum', 'vusage_offnet_max','vusage_offnet_min', 
                 'vusage_offnet_days', 'vusage_offnet_avg','vusage_offnet_stddev', 
                 'number_of_fixed_bb_accounts','number_of_iptv_accounts', 
                 'add_on_tot_rental', 'add_on_count']

In [20]:
X = train_data.drop(columns=['encoded_class_labels', 'primary_identifier'])

In [21]:
y = train_data['encoded_class_labels']

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=10, test_size = 0.05, stratify = y)

In [23]:
X_train.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,...,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
825,4G,Smartphone,FEMALE,Gampaha,20-30,78,1883.00,8098.0,16.484163,1193.490847,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,110.0,1.0
452,4G,Smartphone,MALE,Puttalam,20-30,74,1235.68,4805.0,0.000000,573.887156,...,203.0,36.200000,0.0,24.0,8.468056,8.451039,0.0,0.0,50.0,1.0
3175,4G,Smartphone,FEMALE,Colombo,20-30,54,4901.43,3065.0,0.000000,598.372708,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,160.0,2.0
8908,4G,Smartphone,MALE,Colombo,20-30,78,1442.65,13784.0,115.651798,854.164129,...,10.0,4.716667,0.0,7.0,1.542857,0.973741,0.0,0.0,1490.0,1.0
3484,4G,Smartphone,MALE,Matara,30-40,115,2538.34,6119.0,4.909788,1376.794269,...,4.0,4.116667,0.0,1.0,4.116667,0.751597,0.0,0.0,0.0,0.0


In [24]:
quan_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

quan_transformed = quan_pipeline.fit_transform(X_train[data_quan_cols])

In [25]:
data_pipeline = ColumnTransformer([
    ('numerical', quan_pipeline, data_quan_cols),
    ('categorical', OneHotEncoder(), data_cat_cols),
    
])

train_data_processed = data_pipeline.fit_transform(X_train)

In [26]:
train_data_processed

array([[-0.19976851, -0.13366101, -0.29556044, ...,  0.        ,
         0.        ,  0.        ],
       [-0.25556159, -0.57975143, -0.50298003, ...,  0.        ,
         0.        ,  0.        ],
       [-0.53452702,  1.94644268, -0.6125792 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.24657618, -1.0374262 , -0.80563752, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17614751, -0.75819587,  0.02259211, ...,  0.        ,
         0.        ,  0.        ],
       [-0.75769937, -0.19473209, -0.80563752, ...,  0.        ,
         0.        ,  0.        ]])

In [27]:
X_test.head()

,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,...,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
1085,4G,Smartphone,MALE,Gampaha,>70,197,1779.12,3323.0,0.000000,979.542162,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
363,4G,Smartphone,MALE,Kalutara,30-40,6,3090.40,20448.0,83.657084,1765.653071,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0
3138,4G,Smartphone,MALE,Kalutara,>70,209,1221.33,5969.0,23.828087,457.254642,...,35.0,14.100000,0.0,8.0,4.397917,2.939344,0.0,0.0,0.0,0.0
8803,4G,Smartphone,MALE,Ampara,30-40,110,905.32,5799.0,1.819816,1971.909241,...,8.0,2.583333,0.0,4.0,2.112500,0.741921,0.0,0.0,0.0,0.0
2588,4G,Smartphone,MALE,Colombo,30-40,21,1481.54,26751.0,435.232862,2204.760595,...,1.0,1.116667,0.0,2.0,0.866667,0.229473,0.0,0.0,700.0,3.0


In [28]:
X_train.columns

Index(['device_type', 'device_category', 'gender', 'district_name',
       'age_group', 'network_stay', 'average_monthly_bill_amount',
       'dusage_sum', 'dusage_min', 'dusage_max', 'dusage_avg', 'dusage_days',
       'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_max',
       'vusage_onnet_min', 'vusage_onnet_days', 'vusage_onnet_stddev',
       'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min',
       'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev',
       'number_of_fixed_bb_accounts', 'number_of_iptv_accounts',
       'add_on_tot_rental', 'add_on_count'],
      dtype='object')

In [29]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 517 entries, 1085 to 3521
Data columns (total 28 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  517 non-null    object 
 1   device_category              517 non-null    object 
 2   gender                       517 non-null    object 
 3   district_name                517 non-null    object 
 4   age_group                    517 non-null    object 
 5   network_stay                 517 non-null    int64  
 6   average_monthly_bill_amount  517 non-null    float64
 7   dusage_sum                   517 non-null    float64
 8   dusage_min                   517 non-null    float64
 9   dusage_max                   517 non-null    float64
 10  dusage_avg                   517 non-null    float64
 11  dusage_days                  517 non-null    float64
 12  dusage_stddev                517 non-null    float64
 13  vusage_onnet_sum

In [30]:
test_data_processed = data_pipeline.transform(X_test)

In [31]:
test_data_processed

array([[ 1.4600758 , -0.20524828, -0.59632829, ...,  0.        ,
         0.        ,  1.        ],
       [-1.20404406,  0.69839976,  0.48234176, ...,  0.        ,
         0.        ,  0.        ],
       [ 1.62745506, -0.58964051, -0.42966196, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [-0.71585455, -0.54605965, -0.79429968, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.83240358, -0.73006541, -0.69685719, ...,  0.        ,
         0.        ,  0.        ],
       [-0.71585455, -0.54652137, -0.63487697, ...,  0.        ,
         0.        ,  0.        ]])

In [32]:
model = RandomForestClassifier()

In [33]:
model.fit(train_data_processed, Y_train)

RandomForestClassifier()

In [34]:
y_pred = model.predict(test_data_processed)

In [35]:
print(y_pred)

[5 6 2 1 2 1 3 3 1 1 4 1 5 5 1 5 5 3 6 1 1 7 3 5 1 1 6 1 1 5 1 1 1 3 5 1 4
 5 6 1 1 3 5 1 1 1 3 1 1 5 1 6 1 5 5 3 3 5 1 1 1 1 1 1 1 1 5 6 1 1 1 5 4 4
 1 6 1 1 3 5 1 1 1 1 1 1 3 4 3 3 3 1 1 6 4 1 3 6 1 1 2 6 1 5 1 1 1 1 3 6 1
 5 3 1 5 2 1 1 4 1 3 5 3 1 1 1 8 5 1 1 1 2 3 3 3 1 3 1 1 5 1 1 1 5 1 5 3 1
 5 6 3 8 1 3 1 1 1 1 1 4 3 2 1 6 3 1 5 8 5 1 3 8 4 1 3 1 1 1 1 2 3 4 1 6 5
 1 1 3 5 1 1 3 7 5 1 1 6 3 4 3 5 5 3 3 5 5 1 1 4 3 5 5 5 1 5 3 5 1 2 1 5 1
 5 5 3 3 1 5 5 5 1 5 1 5 1 1 1 3 5 1 3 1 5 3 1 1 1 3 1 5 1 5 1 3 5 1 3 4 1
 1 3 3 3 1 3 6 1 5 1 3 5 3 3 3 1 2 3 1 1 3 1 5 5 1 5 3 1 1 1 3 1 1 1 1 1 1
 5 3 1 3 1 6 1 3 1 1 1 1 4 1 1 4 1 3 3 1 1 3 3 3 3 4 3 3 1 1 4 1 8 1 3 1 3
 3 1 1 3 5 1 8 1 1 3 5 1 3 2 2 3 5 5 5 1 1 1 1 1 3 3 1 3 1 3 1 1 8 1 5 5 5
 5 4 1 3 3 1 5 3 1 5 5 1 7 3 5 5 1 1 5 5 1 3 5 1 3 5 5 3 1 1 6 1 3 1 1 1 6
 1 5 1 1 1 3 5 1 5 1 8 1 1 1 1 3 1 1 1 1 1 5 1 1 5 5 3 3 2 5 1 5 1 1 6 5 6
 1 3 5 5 1 1 1 1 4 1 5 5 2 3 1 3 3 1 1 3 1 1 5 5 3 3 1 1 1 5 7 1 3 3 1 5 1
 2 3 1 6 1 1 3 5 3 5 1 3 

In [36]:
confusion_matrix(Y_test, y_pred)

array([[154,   3,  17,   2,   5,   0,   0,   0],
       [ 46,   7,  10,   4,   0,   0,   0,   0],
       [ 23,   2,  51,   6,   3,   0,   0,   0],
       [  6,   0,  23,   4,   2,   0,   1,   0],
       [  5,   2,  10,   1,  53,   6,   0,   0],
       [  0,   0,   1,   2,  23,   8,   0,   2],
       [  0,   1,   0,   0,  10,   2,   2,   3],
       [  2,   0,   0,   0,   4,   7,   1,   3]])

In [37]:
# Getting f1 score
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.65      0.85      0.74       181
           2       0.47      0.10      0.17        67
           3       0.46      0.60      0.52        85
           4       0.21      0.11      0.15        36
           5       0.53      0.69      0.60        77
           6       0.35      0.22      0.27        36
           7       0.50      0.11      0.18        18
           8       0.38      0.18      0.24        17

    accuracy                           0.55       517
   macro avg       0.44      0.36      0.36       517
weighted avg       0.51      0.55      0.50       517



In [38]:
ran_for_params = {
    'n_estimators' : range(1,100),
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1,10),
    'min_samples_leaf': range(1,10),
    'min_samples_split': range(1,10),
    'bootstrap' : [True, False]
}

In [39]:
random_forest_model = RandomForestClassifier()
model_rand_fors = RandomizedSearchCV(random_forest_model, ran_for_params, cv=5, n_jobs = -1, verbose = 1, n_iter = 1000)
model_rand_fors.fit(train_data_processed, Y_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


/opt/anaconda3/envs/datastorm2/lib/python3.7/site-packages/sklearn/model_selection/_search.py:921: UserWarning: One or more of the test scores are non-finite: [       nan 0.45539715 0.51558045 0.4607943  0.44185336 0.45539715
 0.45549898 0.51812627 0.52606925 0.49144603 0.51527495 0.50977597
 0.47769857 0.48798371 0.52291242 0.48452138 0.44867617 0.51191446
 0.52464358 0.49623218 0.52566191 0.50346232 0.50824847 0.52556008
 0.50824847 0.51843177 0.52179226 0.4509165  0.48289206 0.45203666
 0.52545825 0.53004073 0.45437882 0.50142566 0.4811609  0.45468432
 0.50101833 0.50417515 0.49389002 0.51710794 0.52077393 0.48971487
 0.52077393 0.4212831  0.5188391  0.44460285 0.51670061 0.45702648
 0.48014257 0.51232179 0.45610998 0.47647658 0.47576375 0.47698574
 0.52016293 0.47953157 0.41985743 0.48401222 0.49297352        nan
 0.46568228 0.51720978 0.50661914 0.51160896 0.52698574 0.45600815
 0.53207739 0.450611   0.49470468        nan 0.45570265        nan
        nan 0.45661914 0.50366599    

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=1000,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'criterion': ['gini', 'entropy'],
                                        'max_depth': range(1, 10),
                                        'min_samples_leaf': range(1, 10),
                                        'min_samples_split': range(1, 10),
                                        'n_estimators': range(1, 100)},
                   verbose=1)

In [40]:
model_rand_fors.best_params_

{'n_estimators': 94,
 'min_samples_split': 4,
 'min_samples_leaf': 4,
 'max_depth': 9,
 'criterion': 'entropy',
 'bootstrap': False}

In [41]:
model_rand_fors.best_score_

0.5320773930753564

In [42]:
tuned_model = RandomForestClassifier(n_estimators = 94, min_samples_split = 4,min_samples_leaf =  4,
                                     max_depth = 9, criterion = 'entropy', bootstrap = False)

In [43]:
tuned_model.fit(train_data_processed, Y_train)

RandomForestClassifier(bootstrap=False, criterion='entropy', max_depth=9,
                       min_samples_leaf=4, min_samples_split=4,
                       n_estimators=94)

In [44]:
y_pred = tuned_model.predict(test_data_processed)

In [45]:
# Getting f1 score
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.60      0.86      0.70       181
           2       1.00      0.01      0.03        67
           3       0.41      0.58      0.48        85
           4       0.00      0.00      0.00        36
           5       0.50      0.77      0.61        77
           6       0.11      0.03      0.04        36
           7       0.50      0.06      0.10        18
           8       0.38      0.18      0.24        17

    accuracy                           0.52       517
   macro avg       0.44      0.31      0.28       517
weighted avg       0.52      0.52      0.43       517

